In [3]:
# Import necessary libraries
from google.cloud import bigquery
import pandas as pd
import os
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("service_account_key.json")

# Initialize a BigQuery client
client = bigquery.Client(
    credentials=credentials
)

# Define your SQL query
query = """
    SELECT *
    FROM `dev_gsokolov.otr_counts`
"""

# Execute the query and convert the result to a pandas DataFrame
df = client.query(query).to_dataframe()

In [4]:
df

,user_id,account_id,deal_count,total_reading_count
0,34412973,<NA>,0,23
1,13468718,101173420,38,790
2,37577676,100793821,756,45
3,251726,121701726,142,107
4,39082807,121893091,72,33
...,...,...,...,...
171588,2259969,<NA>,0,16
171589,14445781,<NA>,0,16
171590,40099057,142059404,54,16
171591,4847860,120036766,122,16


In [ ]:
df.to_csv('otr_counts.csv', index=False)
# cut values of total_trading_count into predefined bins
bins = [0,100,250,500,750,1000,1500,2000,3000,4000,5000]
labels = ['0-100','100-250','250-500','500-750','750-1000','1000-1500','1500-2000','2000-3000','3000-4000','4000-5000']
df['total_trading_count_binned'] = pd.cut(df['total_trading_count'], bins=bins, labels=labels)

In [9]:
filtered_df = df[(df['total_reading_count'] < 1000) & (df['total_reading_count'] != 0)]

# Define bin edges
bin_edges = [0, 1, 2, 3, 4, 5, 6, 10, 20, 50, 100, filtered_df['total_reading_count'].max() + 1]

# Define bin labels
bin_labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-10', '10-20', '20-50', '50-100', '100+']

# Assign bins to the total_reading_count column
filtered_df['reading_count_bin'] = pd.cut(filtered_df['total_reading_count'], bins=bin_edges, labels=bin_labels, right=False, include_lowest=True)

filtered_df


/var/folders/n0/lll36rgn2xdcdg6twr3671vmm2ffcc/T/ipykernel_63774/265316934.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['reading_count_bin'] = pd.cut(filtered_df['total_reading_count'], bins=bin_edges, labels=bin_labels, right=False, include_lowest=True)


,user_id,account_id,deal_count,total_reading_count,reading_count_bin
0,40092914,101134895,156,26,20-50
1,13054545,121291660,311,33,20-50
2,564873,<NA>,0,19,10-20
3,7990330,<NA>,0,21,20-50
4,9858381,<NA>,0,30,20-50
...,...,...,...,...,...
171588,6942831,<NA>,0,16,10-20
171589,34081549,<NA>,0,16,10-20
171590,31547929,<NA>,0,16,10-20
171591,33313478,121191739,4,16,10-20


In [17]:
df[df['deal_count']>11]

,user_id,account_id,deal_count,total_reading_count
0,40092914,101134895,156,26
1,13054545,121291660,311,33
5,5775527,101154296,62,854
11,17813330,100877125,351,97
12,17813330,100877124,28,97
...,...,...,...,...
171578,36950141,140964313,164,16
171580,40267373,122348315,26,16
171583,38778472,121763466,136,16
171586,40090520,122283503,300,16


In [15]:
import matplotlib.pyplot as plt

# Assuming `df` is your DataFrame and it contains a column named `total_reading_count`
# Filter the DataFrame to include only rows where `total_reading_count` is less than 1000
filtered_df = df[(df['deal_count'] < 100) & (df['deal_count'] != 0)]
display(filtered_df)

,user_id,account_id,deal_count,total_reading_count
5,5775527,101154296,62,854
12,17813330,100877124,28,97
16,13944665,100742044,78,478
17,40248703,101183843,38,63
20,39347883,141803796,52,51
...,...,...,...,...
171569,12488677,121234912,8,16
171571,4035355,120820955,12,16
171576,22157491,101152451,28,16
171580,40267373,122348315,26,16


In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# Create a binary target variable: 1 if user made at least 5 deals, 0 otherwise
df['made_at_least_5_deals'] = (df['deal_count'] >= 5).astype(int)

# Fit logistic regression model
X = df[['deal_count']]
y = df['made_at_least_5_deals']
model = LogisticRegression()
model.fit(X, y)

# Predict the probability of making at least 5 deals for each user
df['probability_of_at_least_5_deals'] = model.predict_proba(X)[:, 1]

# Since winning is certain if at least 5 deals are made, probability of winning is the same
df['probability_of_winning'] = df['probability_of_at_least_5_deals']

# Calculate the overall probability of winning
overall_probability_of_winning = df['probability_of_winning'].mean()

display(df)
print(f"Overall Probability of Winning: {overall_probability_of_winning}")

,user_id,deal_count,made_at_least_5_deals,probability_of_at_least_5_deals,probability_of_winning
0,16798787,106927,1,1.000000e+00,1.000000e+00
1,12306957,47717,1,1.000000e+00,1.000000e+00
2,909086,21773,1,1.000000e+00,1.000000e+00
3,35716982,13252,1,1.000000e+00,1.000000e+00
4,25163307,12166,1,1.000000e+00,1.000000e+00
...,...,...,...,...,...
78169,22690424,1,0,2.130930e-16,2.130930e-16
78170,18831466,1,0,2.130930e-16,2.130930e-16
78171,10455219,1,0,2.130930e-16,2.130930e-16
78172,25987599,1,0,2.130930e-16,2.130930e-16


Overall Probability of Winning: 0.8734617647812162


In [8]:
# Given constants
gift_price = 10  # Example gift price
email_open_rate = 0.2  # Example probability of opening the email
email_click_rate = 0.01  # Example probability of clicking the promotion

# Calculate the probability of winning adjusted for the email open rate and click rate
df['adjusted_probability_of_winning'] = df['probability_of_at_least_5_deals'] * email_open_rate * email_click_rate

# Calculate expected spending for each user
df['expected_spending'] = df['adjusted_probability_of_winning'] * gift_price

# Calculate total expected spending
total_expected_spending = df['expected_spending'].sum()

In [9]:
total_expected_spending

1365.6400000001358

In [10]:
from scipy.stats import norm
# Calculate the 95% confidence interval lower bound for each probability
# Predict the probability of making at least 5 deals for each user
prob_predictions = model.predict_proba(X)[:, 1]

confidence_level = 0.95
z_score = norm.ppf((1 + confidence_level) / 2)
std_error = np.sqrt((prob_predictions * (1 - prob_predictions)) / len(df))  # standard error of probability
lower_bounds = prob_predictions - z_score * std_error

In [11]:
lower_bounds

array([ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
       -1.02329437e-10, -1.02329437e-10, -1.02329437e-10])

In [14]:
df['adjusted_probability_of_winning_lower_bound'] = lower_bounds * email_open_rate * email_click_rate
# Calculate expected spending for each user using the lower bounds
df['expected_spending_lower_bound'] = df['adjusted_probability_of_winning_lower_bound'] * gift_price
total_expected_spending_lower_bound = df['expected_spending_lower_bound'].sum()
total_expected_spending_lower_bound

1365.6020673399892

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm

# Define your SQL query
query = """
    SELECT *
    FROM `dev_gsokolov.deal_cnt_tw`
"""

# Execute the query and convert the result to a pandas DataFrame
df = client.query(query).to_dataframe()

# Create a binary target variable: 1 if user made at least 5 deals, 0 otherwise
df['made_at_least_5_deals'] = (df['deal_count'] >= 5).astype(int)

# Fit logistic regression model
X = df[['deal_count']]
y = df['made_at_least_5_deals']
model = LogisticRegression()
model.fit(X, y)

# Predict the probability of making at least 5 deals for each user
prob_predictions = model.predict_proba(X)[:, 1]

# Calculate the 95% confidence interval lower bound for each probability
confidence_level = 0.95
z_score = norm.ppf((1 + confidence_level) / 2)
std_error = np.sqrt((prob_predictions * (1 - prob_predictions)) / len(df))  # standard error of probability
lower_bounds = prob_predictions - z_score * std_error

# Replace negative probabilities with zero (since probabilities cannot be negative)
lower_bounds = np.maximum(lower_bounds, 0)

# Given constants
gift_price = 10  # Example gift price
email_open_rate = 0.2  # Example probability of opening the email
email_click_rate = 0.01  # Example probability of clicking the promotion

# Calculate the probability of winning adjusted for the email open rate and click rate using the lower bounds
df['adjusted_probability_of_winning_lower_bound'] = lower_bounds * email_open_rate * email_click_rate

# Calculate expected spending for each user using the lower bounds
df['expected_spending_lower_bound'] = df['adjusted_probability_of_winning_lower_bound'] * gift_price

# Calculate total expected spending using the lower bounds
total_expected_spending_lower_bound = df['expected_spending_lower_bound'].sum()

display(df)
print(f"Total Expected Spending on Gifts (Lower Bound): {total_expected_spending_lower_bound:.2f}")

,user_id,deal_count,made_at_least_5_deals,adjusted_probability_of_winning_lower_bound,expected_spending_lower_bound
0,16798787,78193,1,0.002,0.02
1,12306957,32491,1,0.002,0.02
2,909086,12944,1,0.002,0.02
3,7545452,10196,1,0.002,0.02
4,25163307,9924,1,0.002,0.02
...,...,...,...,...,...
78169,4509711,0,0,0.000,0.00
78170,22948052,0,0,0.000,0.00
78171,15432788,0,0,0.000,0.00
78172,30464031,0,0,0.000,0.00


Total Expected Spending on Gifts (Lower Bound): 1325.66


In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm

# Define your SQL query
query = """
    SELECT *
    FROM `dev_gsokolov.tw_segment_space_event_counts`
"""

# Execute the query and convert the result to a pandas DataFrame
data = client.query(query).to_dataframe()

In [21]:
data

,user_id,app_event_count_month,app_event_count_year,web_event_count_month,web_event_count_year,total_event_count_month,total_event_count_year
0,383513,18,18,26,26,44,44
1,22662788,0,81,0,0,0,81
2,12239479,0,27,0,0,0,27
3,40116410,121,121,0,0,121,121
4,33828138,0,8,0,44,0,52
...,...,...,...,...,...,...,...
156840,26570291,0,16,0,0,0,16
156841,30779468,12,16,0,0,12,16
156842,28748152,5,16,0,0,5,16
156843,3043962,10,16,0,0,10,16


In [19]:
# Filter users who have not read any articles this month
df = df[df['total_event_count_month'] == 0]

# Create a binary target variable: 1 if user reads at least 10 articles in a month, 0 otherwise based on yearly data
historical_df = pd.DataFrame(data)
historical_df['read_at_least_10_articles'] = (historical_df['total_event_count_month'] >= 10).astype(int)

# Fit logistic regression model using yearly data to predict monthly behavior
X = historical_df[['total_event_count_year']]
y = historical_df['read_at_least_10_articles']
model = LogisticRegression()
model.fit(X, y)

# Predict the probability of reading at least 10 articles in a month for each user based on yearly data
df['probability_of_at_least_10_articles'] = model.predict_proba(df[['total_event_count_year']])[:, 1]

# Given constants
gift_price = 10  # Gift price
email_open_rate = 0.2  # Email open rate
email_click_rate = 0.01  # Email click rate

# Calculate the probability of winning adjusted for the email open rate and click rate
df['adjusted_probability_of_winning'] = df['probability_of_at_least_10_articles'] * email_open_rate * email_click_rate

# Calculate expected spending for each user
df['expected_spending'] = df['adjusted_probability_of_winning'] * gift_price

# Calculate total expected spending
total_expected_spending = df['expected_spending'].sum()

# Calculate the 95% confidence interval lower bound for each probability
confidence_level = 0.95
z_score = norm.ppf((1 + confidence_level) / 2)
std_error = np.sqrt((df['probability_of_at_least_10_articles'] * (1 - df['probability_of_at_least_10_articles'])) / len(df))  # standard error of probability
lower_bounds = df['probability_of_at_least_10_articles'] - z_score * std_error

# Replace negative probabilities with zero (since probabilities cannot be negative)
lower_bounds = np.maximum(lower_bounds, 0)

# Calculate the probability of winning adjusted for the email open rate and click rate using the lower bounds
df['adjusted_probability_of_winning_lower_bound'] = lower_bounds * email_open_rate * email_click_rate

# Calculate expected spending for each user using the lower bounds
df['expected_spending_lower_bound'] = df['adjusted_probability_of_winning_lower_bound'] * gift_price

# Calculate total expected spending using the lower bounds
total_expected_spending_lower_bound = df['expected_spending_lower_bound'].sum()

total_expected_spending, total_expected_spending_lower_bound

(266.34077491644535, 262.24393225607497)

In [26]:
data

,user_id,app_event_count_month,app_event_count_year,web_event_count_month,web_event_count_year,total_event_count_month,total_event_count_year
0,383513,18,18,26,26,44,44
1,22662788,0,81,0,0,0,81
2,12239479,0,27,0,0,0,27
3,40116410,121,121,0,0,121,121
4,33828138,0,8,0,44,0,52
...,...,...,...,...,...,...,...
156840,26570291,0,16,0,0,0,16
156841,30779468,12,16,0,0,12,16
156842,28748152,5,16,0,0,5,16
156843,3043962,10,16,0,0,10,16


In [35]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

df = pd.DataFrame(data)

# Calculate total event counts as per your SQL logic
df['total_event_count_month'] = df['app_event_count_month'] + df['web_event_count_month']
df['total_event_count_year'] = df['app_event_count_year'] + df['web_event_count_year']

# Function to calculate the probability of reading more than 10 articles
def calculate_probability(lambda_value):
    # P(X > 10) = 1 - P(X <= 10)
    return 1 - poisson.cdf(10, lambda_value)

# Estimate the Poisson parameter (λ) for each user based on the current month's event count
df['lambda'] = df['total_event_count_month']

# Calculate the probability for each user
df['probability_more_than_10'] = df['lambda'].apply(calculate_probability)

df


,user_id,app_event_count_month,app_event_count_year,web_event_count_month,web_event_count_year,total_event_count_month,total_event_count_year,lambda,probability_more_than_10
0,383513,18,18,26,26,44,44,44,1.000000
1,22662788,0,81,0,0,0,81,0,0.000000
2,12239479,0,27,0,0,0,27,0,0.000000
3,40116410,121,121,0,0,121,121,121,1.000000
4,33828138,0,8,0,44,0,52,0,0.000000
...,...,...,...,...,...,...,...,...,...
156840,26570291,0,16,0,0,0,16,0,0.000000
156841,30779468,12,16,0,0,12,16,12,0.652771
156842,28748152,5,16,0,0,5,16,5,0.013695
156843,3043962,10,16,0,0,10,16,10,0.416960


In [36]:
# Define the cost of the gift
gift_cost = 10

# Calculate the expected cost for each user
df['expected_cost'] = df['probability_more_than_10'] * gift_cost

# Calculate the total money spent
total_money_spent = df['expected_cost'].sum()

In [37]:
total_money_spent

320545.6111444049

In [38]:
# Calculate total event counts as per your SQL logic
df['total_event_count_month'] = df['app_event_count_month'] + df['web_event_count_month']
df['total_event_count_year'] = df['app_event_count_year'] + df['web_event_count_year']

# Function to calculate the probability of reading more than 10 articles
def calculate_probability(lambda_value):
    # P(X > 10) = 1 - P(X <= 10)
    return 1 - poisson.cdf(10, lambda_value)

# Estimate the Poisson parameter (λ) for each user based on the current month's event count
df['lambda'] = df['total_event_count_month']

# Calculate the probability for each user
df['probability_more_than_10'] = df['lambda'].apply(calculate_probability)

Total money spent during the promotion: $64109.12


In [39]:
# Adjust the probability with the email open rate (20%) and email click rate (1%)
email_open_rate = 0.2
email_click_rate = 0.01
df['adjusted_probability_more_than_10'] = df['probability_more_than_10'] * email_open_rate * email_click_rate

# Define the cost of the gift
gift_cost = 10

# Calculate the expected cost for each user
df['expected_cost'] = df['adjusted_probability_more_than_10'] * gift_cost

# Calculate the total money spent
total_money_spent = df['expected_cost'].sum()

Total money spent during the promotion: $641.09
